In [ ]:
# pip install ipympl
import os
import cv2
import numpy as np
import pandas as pd
from skimage import measure, morphology
from skimage.io import imread
from tkinter import filedialog, Tk
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from scipy.ndimage import convolve
plt.rcParams['figure.figsize'] = [8, 8]

GUI = False

# Set path and min-max paramters
if GUI:
    Tk().withdraw()
    input_folder = filedialog.askdirectory(title="Choose input folder")
    output_file = filedialog.asksaveasfilename(defaultextension=".csv", title="Choose where to save results", initialfile="Intensity_Results.csv")
    min_val = float(input("Enter intensity minimum (e.g. 0): "))
    max_val = float(input("Enter intensity maximum (e.g. 16000): "))
else:
    input_folder = os.getcwd() # Current folder
    output_file = os.path.join(input_folder, "Intensity_Results.csv")
    min_val = 0
    max_val = 16000

# Collect image files
files = sorted(os.listdir(input_folder))
filtered_files = [f for f in files if f.endswith("tiff")]

# Save image files to list
images = []
for j in range(0, len(filtered_files), 2):
    int_path = os.path.join(input_folder, filtered_files[j])
    nuc_path = os.path.join(input_folder, filtered_files[j+1])
    int_img = imread(int_path, as_gray=True)
    nuc_img = imread(nuc_path, as_gray=True)

    # Scale intensity image
    int_img = (np.clip(int_img, min_val, max_val) - min_val) / (max_val - min_val)
    images.append([filtered_files[j].split(".")[0], int_img, nuc_img])

In [ ]:
%matplotlib inline
def GUI_binary(entry):
    name = entry[0]
    nuc_img = entry[2]
    slider_thresh = widgets.IntSlider(value=500, min=0, max=2500, step=10, description="Threshold value:", continuous_update=True)
    slider_minsize = widgets.IntSlider(value=500, min=0, max=1000, step=10, description="Min size:", continuous_update=True)
    button = widgets.Button(description="Confirm", button_style="success")
    output = widgets.Output()

    plt.close()
    plt.figure()

    def get_binary(thresh_val, min_size):
        binary = (nuc_img > thresh_val).astype(np.uint8)
        binary = morphology.remove_small_objects(binary.astype(bool), min_size=min_size)
        return binary

    def update_image(change=None):
        with output:
            binary = get_binary(slider_thresh.value, slider_minsize.value)
            clear_output(wait=True)
            plt.title(name)
            plt.axis('off')
            plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
            plt.imshow(binary , cmap='gray')
            plt.show()

    def on_confirm_clicked(b=None):
        """Store the confirmed value and clear the plot"""
        binary = get_binary(slider_thresh.value, slider_minsize.value)
        plt.close()
        container.close()
        display(f"Confirmed values for \"{name}\": thresh_val = {slider_thresh.value}, min_size = {slider_minsize.value}")
        if (len(entry) == 3):
            entry.append(binary)
        else:
            entry[3] = binary

    slider_thresh.observe(update_image, names="value")
    slider_minsize.observe(update_image, names="value")
    container = widgets.VBox([slider_thresh, slider_minsize, button, output])
    display(container)
    update_image()
    button.on_click(on_confirm_clicked)

for entry in images:
    GUI_binary(entry)


In [ ]:
expand_size = 1.5
shrink_size = 0.5
intensity_thresh = 0.37
activity_fraction_thresh = 0.1

intensity_thresh_ALT = 0.15
activity_fraction_thresh_ALT = 0.1

results = []
all_perinuclear_list = []
all_perinuclear_active_list = []
all_perinuclear_inactive_list = []
for entry in images:
    name, int_img, nuc_img, binary = entry
    all_perinuclear = False
    all_perinuclear_active = False
    all_perinuclear_inactive = False
    active = 0
    inactive = 0

    # Label nuclei
    labels = measure.label(binary)
    props = measure.regionprops(labels)

    for idx, prop in enumerate(props, start=1):
        mask = (labels == prop.label).astype(np.uint8)

        # Perinuclear region = expand - shrink
        shrink = cv2.erode(mask, np.ones((int(shrink_size*2+1), int(shrink_size*2+1)), np.uint8), iterations=1)
        expand = cv2.dilate(mask, np.ones((int(expand_size*2+1), int(expand_size*2+1)), np.uint8), iterations=1)
        perinuclear = ((expand - shrink) > 0).astype(bool)

        # Get values and measure intensity inside perinuclear mask
        values = int_img[perinuclear]
        mean_intensity = values.mean()
        all_perinuclear = np.logical_or(all_perinuclear, perinuclear) 

        if True:
            '''Classify according to number of dark pixels'''
            # Determine fraction of active pixels and set ROI to active or inactive accordingly
            proportion_active_pixels = int(np.sum(values > intensity_thresh)) / len(values)
            if proportion_active_pixels > activity_fraction_thresh:
                # Cell is active
                active += 1
                all_perinuclear_active = np.logical_or(all_perinuclear_active, perinuclear)
            else:
                # Cell is inactive
                inactive += 1
                all_perinuclear_inactive = np.logical_or(all_perinuclear_inactive, perinuclear)
        else:
            '''Classify according to number of dark pixels and their neighbors'''
            dark_pixels = (int_img <= intensity_thresh_ALT)
            dark_pixels_masked = dark_pixels & perinuclear
            kernel = np.array([[1,1,1],[1,0,1],[1,1,1]])
            neighbor_count = convolve(dark_pixels_masked.astype(int), kernel, mode='constant', cval=0)
            surrounded = (neighbor_count >= 5) & dark_pixels_masked
            if int(np.sum(surrounded)) < activity_fraction_thresh_ALT:
                # Cell is active
                active += 1
                all_perinuclear_active = np.logical_or(all_perinuclear_active, perinuclear)
            else:
                # Cell is inactive
                inactive += 1
                all_perinuclear_inactive = np.logical_or(all_perinuclear_inactive, perinuclear)

        results.append({
            "ID": idx,
            "MeanIntensity": mean_intensity,
            "Name": name,
            "Values": np.array2string(values, separator=',', max_line_width=np.inf).replace(' ', '')
        })
    
    all_perinuclear_list.append(all_perinuclear)
    all_perinuclear_active_list.append(all_perinuclear_active)
    all_perinuclear_inactive_list.append(all_perinuclear_inactive)
    print(f"\"{name}\": Active: {active}, inactive: {inactive}. Percentage active: {100.0 * active / (active + inactive) :.1f}%.")

pd.DataFrame(results).to_csv(output_file, index=False, mode="w")
print(f"Analysis complete! Results saved to {output_file}.")


In [ ]:
%matplotlib widget
plt.close()
for i in range(len(images)):
    int_img = images[i][1]
    plt.figure()
    plt.axis('off')
    plt.title(images[i][0])
    plt.imshow(int_img, cmap='gray')
    contours_active = measure.find_contours(all_perinuclear_active_list[i], level=0.5)
    contours_inactive = measure.find_contours(all_perinuclear_inactive_list[i], level=0.5)
    for contour in contours_active:
        plt.plot(contour[:, 1], contour[:, 0], 'g-', linewidth=0.5)
    for contour in contours_inactive:
        plt.plot(contour[:, 1], contour[:, 0], 'r-', linewidth=0.5)
    plt.show()